In [1]:
import os

if "run_molecular_property_based_filtering.py" not in os.listdir():
    os.chdir("../src")

os.listdir()

['database',
 'utils',
 'screening',
 'run_molecular_property_based_filtering.py',
 '__init__.py']

---

In [2]:
import pandas as pd

In [9]:
df = pd.read_csv(
    "../data/chembl_34_initial_screening_results.csv", sep=",", index_col=0
)

In [10]:
display(df.head())
df.shape

,Lipinski_Rule_of_5,Ghose_Filter,Veber_Filter,Rule_of_3_Filter,REOS_Filter,Drug_like_Filter,QED_Filter,TPSA_Filter,SA_Score_Filter,NP_Score_Filter,Errors
ChemblID,,,,,,,,,,,
CHEMBL153534,True,False,True,False,True,True,0.608781,82.22,3.048474,-1.552303,[]
CHEMBL440060,False,False,False,False,False,False,0.016357,1271.93,9.084590,-0.037746,[]
CHEMBL440245,False,False,False,False,False,False,0.010798,1502.95,9.752879,0.219818,[]
CHEMBL440249,False,False,False,False,False,False,0.016726,979.36,9.146251,0.632360,[]
CHEMBL405398,False,True,True,False,True,False,0.623114,75.20,2.595185,-1.628093,[]


(335452, 11)

(334969, 11)


In [12]:
"CHEMBL130273" in set(df.index)

True